In [1]:
import os
import sys
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from graphframes import  *
import json
import itertools
import math
import time

In [2]:
appName = 'assignment4'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master).set('spark.jars.packages','graphframes:graphframes:0.6.0-spark2.3-s_2.11')
conf = conf.setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '3'), ('spark.cores.max', '3'), ('spark.driver.memory','8g')])
sc = SparkContext(conf=conf)
sc.setLogLevel("INFO")
sqlContext = SQLContext(sc)
sc

<SparkContext master=local[*] appName=assignment4>

In [3]:
input_path = './data/power_input.txt'
edges = sc.textFile(input_path).map(lambda x: tuple(x.split())).collect()
vertices = sc.textFile(input_path).flatMap(lambda x: x.split()).distinct().collect()
vertices = sqlContext.createDataFrame([(x,) for x in vertices], ['id'])
edges = sqlContext.createDataFrame(edges, ['src', 'dst'])

In [4]:
graph = GraphFrame(vertices, edges)

In [13]:
result = graph.labelPropagation(maxIter=5)

In [14]:
result = result.rdd.map(lambda x: (x['label'], x['id'])).groupByKey().map(lambda x: (x[0], sorted(list(x[1]))))

In [18]:
length_communities = result.count()

In [19]:
result_sorted = result.takeOrdered(length_communities, lambda x: len(x[1]))

In [21]:
def save_output(output_path, output):
    file = open(output_path, 'wt')
    for _, community in output:
        line = ','.join(community)+'\n'
        file.write(line)

In [22]:
save_output('./data/output1.txt', result_sorted)